In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.model_selection import train_test_split
import random

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Define mango variation classes based on your folder structure
mango_variations = ['Cubed,Hedgehog', 'Sliced,Peeled', 'Whole']
num_classes = len(mango_variations)

print(f"Mango Variation Classification")
print(f"Number of classes: {num_classes}")
print(f"Class names: {mango_variations}")

# Set data directory path (specifically for Mango folder)
data_dir = r"C:\Users\ashsh\Downloads"
mango_folder = os.path.join(data_dir, "Mango")

print(f"Mango data directory: {mango_folder}")

# Explore mango folder structure and count images
def explore_mango_variations(mango_path, variations):
    total_images = 0
    variation_counts = {}
    
    print(f"\n=== MANGO VARIATIONS ===")
    
    if os.path.exists(mango_path):
        for variation in variations:
            variation_path = os.path.join(mango_path, variation)
            if os.path.exists(variation_path):
                images = [f for f in os.listdir(variation_path) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                count = len(images)
                variation_counts[variation] = count
                total_images += count
                print(f"{variation}: {count} images")
            else:
                print(f"Warning: {variation} folder not found at {variation_path}")
                variation_counts[variation] = 0
    else:
        print(f"Error: Mango folder not found at {mango_path}")
    
    print(f"\n=== SUMMARY ===")
    print(f"Total mango images: {total_images}")
    
    return variation_counts, total_images

# Explore the mango data
variation_counts, total_images = explore_mango_variations(mango_folder, mango_variations)

# Check if we have sufficient data for each class
min_images_per_class = 100  # Minimum recommended images per class
print(f"\n=== DATA QUALITY CHECK ===")
for variation, count in variation_counts.items():
    status = "✓ Good" if count >= min_images_per_class else "⚠ Low"
    print(f"{variation}: {count} images - {status}")

if total_images > 0:
    print(f"\nReady to proceed with mango variation classification!")
    print(f"This will be a {num_classes}-class classification problem.")
else:
    print(f"\nError: No mango images found. Please check the folder structure.")

Using device: cuda
Mango Variation Classification
Number of classes: 3
Class names: ['Cubed,Hedgehog', 'Sliced,Peeled', 'Whole']
Mango data directory: C:\Users\ashsh\Downloads\Mango

=== MANGO VARIATIONS ===
Cubed,Hedgehog: 1006 images
Sliced,Peeled: 1509 images
Whole: 1019 images

=== SUMMARY ===
Total mango images: 3534

=== DATA QUALITY CHECK ===
Cubed,Hedgehog: 1006 images - ✓ Good
Sliced,Peeled: 1509 images - ✓ Good
Whole: 1019 images - ✓ Good

Ready to proceed with mango variation classification!
This will be a 3-class classification problem.


In [3]:
# COMPLETE MANGO VARIATION CLASSIFIER
# Run this entire code block

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.model_selection import train_test_split
import random
import time
from torch.cuda.amp import GradScaler, autocast

# Set device and random seeds
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

print(f"Using device: {device}")

# Define mango variation classes
mango_variations = ['Cubed,Hedgehog', 'Sliced,Peeled', 'Whole']
num_classes = len(mango_variations)
mango_folder = r"C:\Users\ashsh\Downloads\Mango"

print(f"Mango Variation Classification")
print(f"Number of classes: {num_classes}")
print(f"Class names: {mango_variations}")

# Custom Dataset class for mango variations
class MangoVariationDataset(Dataset):
    def __init__(self, mango_folder, variations, transform=None, train=True, train_split=0.8):
        self.mango_folder = mango_folder
        self.variations = variations
        self.transform = transform
        self.class_to_idx = {variation: idx for idx, variation in enumerate(variations)}
        
        self.image_paths = []
        self.labels = []
        
        print(f"Creating {'train' if train else 'test'} dataset...")
        
        for variation in variations:
            variation_folder = os.path.join(mango_folder, variation)
            if os.path.exists(variation_folder):
                images_in_variation = []
                for img_name in os.listdir(variation_folder):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(variation_folder, img_name)
                        images_in_variation.append(img_path)
                
                train_imgs, test_imgs = train_test_split(images_in_variation, 
                                                       train_size=train_split, 
                                                       random_state=42)
                
                if train:
                    selected_images = train_imgs
                else:
                    selected_images = test_imgs
                
                self.image_paths.extend(selected_images)
                self.labels.extend([self.class_to_idx[variation]] * len(selected_images))
                
                print(f"  {variation}: {len(selected_images)} images")
        
        print(f"Total {'train' if train else 'test'} images: {len(self.image_paths)}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            image = Image.new('RGB', (224, 224), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Data transforms
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
print("Creating mango variation datasets...")
train_dataset = MangoVariationDataset(mango_folder, mango_variations, 
                                     transform=train_transforms, train=True)
test_dataset = MangoVariationDataset(mango_folder, mango_variations, 
                                    transform=test_transforms, train=False)

# Create DataLoaders (no workers)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"DataLoaders created: batch_size={batch_size}")

# Test batch loading
print("Testing batch loading...")
sample_batch = next(iter(train_loader))
print(f"✓ Batch test successful! Images: {sample_batch[0].shape}, Labels: {sample_batch[1].shape}")

# Model class
class MangoVariationClassifier(nn.Module):
    def __init__(self, num_classes=3, pretrained=True, freeze_backbone=True):
        super(MangoVariationClassifier, self).__init__()
        self.resnet = models.resnet18(pretrained=pretrained)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
        
        if freeze_backbone:
            for param in self.resnet.parameters():
                param.requires_grad = False
            for param in self.resnet.fc.parameters():
                param.requires_grad = True
            print("Backbone frozen - only training final layer")
        
    def forward(self, x):
        return self.resnet(x)

# Create model
print("Creating ResNet model for mango variations...")
model = MangoVariationClassifier(num_classes=num_classes, pretrained=True, freeze_backbone=True)
model = model.to(device)

# Print parameter info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)
scaler = GradScaler()

# Training function
def train_mango_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10):
    print(f"Starting training for {num_epochs} epochs...")
    print("-" * 60)
    
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [], 'learning_rates': []}
    best_test_acc = 0.0
    best_model_state = None
    patience_counter = 0
    early_stop_patience = 5
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
            if (batch_idx + 1) % 15 == 0:
                print(f"  Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_corrects.double() / len(train_loader.dataset)
        
        # Testing phase
        model.eval()
        test_running_loss = 0.0
        test_running_corrects = 0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                with autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
                _, preds = torch.max(outputs, 1)
                test_running_loss += loss.item() * inputs.size(0)
                test_running_corrects += torch.sum(preds == labels.data)
        
        test_loss = test_running_loss / len(test_loader.dataset)
        test_acc = test_running_corrects.double() / len(test_loader.dataset)
        
        scheduler.step(test_acc)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Save metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc.item())
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc.item())
        history['learning_rates'].append(current_lr)
        
        # Save best model
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Print results
        epoch_time = time.time() - epoch_start_time
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
        print(f"LR: {current_lr:.6f}, Time: {epoch_time:.1f}s")
        print(f"Best Test Acc: {best_test_acc:.4f}")
        
        # Early stopping
        if patience_counter >= early_stop_patience:
            print(f"Early stopping - no improvement for {early_stop_patience} epochs")
            break
            
        print("-" * 60)
    
    # Load best model
    model.load_state_dict(best_model_state)
    print(f"Training completed! Best test accuracy: {best_test_acc:.4f}")
    return model, history

# Start training
print("Starting mango variation training...")
num_epochs = 10

trained_model, training_history = train_mango_model(
    model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs
)

# Save model
os.makedirs("saved_models", exist_ok=True)
model_save_path = "saved_models/mango_variation_classifier.pth"

torch.save({
    'model_state_dict': trained_model.state_dict(),
    'class_names': mango_variations,
    'num_classes': num_classes,
    'model_architecture': 'resnet18',
    'best_test_accuracy': max(training_history['test_acc']),
    'training_history': training_history
}, model_save_path)

print(f"\n=== TRAINING COMPLETED ===")
print(f"Model saved to: {model_save_path}")
print(f"Best test accuracy: {max(training_history['test_acc']):.4f}")
print(f"Classes: {mango_variations}")

# Function to load the model later
def load_mango_model(model_path, device):
    checkpoint = torch.load(model_path, map_location=device)
    model = MangoVariationClassifier(num_classes=checkpoint['num_classes'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    return model, checkpoint['class_names'], checkpoint['training_history']

print(f"\nTo load this model later, use:")
print(f"model, class_names, history = load_mango_model('{model_save_path}', device)")

Using device: cuda
Mango Variation Classification
Number of classes: 3
Class names: ['Cubed,Hedgehog', 'Sliced,Peeled', 'Whole']
Creating mango variation datasets...
Creating train dataset...
  Cubed,Hedgehog: 804 images
  Sliced,Peeled: 1207 images
  Whole: 815 images
Total train images: 2826
Creating test dataset...
  Cubed,Hedgehog: 202 images
  Sliced,Peeled: 302 images
  Whole: 204 images
Total test images: 708
DataLoaders created: batch_size=64
Testing batch loading...
✓ Batch test successful! Images: torch.Size([64, 3, 224, 224]), Labels: torch.Size([64])
Creating ResNet model for mango variations...
Backbone frozen - only training final layer
Total parameters: 11,178,051
Trainable parameters: 1,539
Starting mango variation training...
Starting training for 10 epochs...
------------------------------------------------------------
Epoch 1/10


C:\Users\ashsh\AppData\Local\Temp\ipykernel_36712\36617082.py:162: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\ashsh\AppData\Local\Temp\ipykernel_36712\36617082.py:190: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  Batch 15/45, Loss: 0.8452
  Batch 30/45, Loss: 0.6463
  Batch 45/45, Loss: 0.4557


C:\Users\ashsh\AppData\Local\Temp\ipykernel_36712\36617082.py:217: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.8142, Train Acc: 0.6617
Test Loss: 0.4978, Test Acc: 0.8729
LR: 0.001000, Time: 667.7s
Best Test Acc: 0.8729
------------------------------------------------------------
Epoch 2/10
  Batch 15/45, Loss: 0.5970
  Batch 30/45, Loss: 0.5204
  Batch 45/45, Loss: 0.2350
Train Loss: 0.4506, Train Acc: 0.8652
Test Loss: 0.3877, Test Acc: 0.8884
LR: 0.001000, Time: 875.1s
Best Test Acc: 0.8884
------------------------------------------------------------
Epoch 3/10
  Batch 15/45, Loss: 0.4005
  Batch 30/45, Loss: 0.2733
  Batch 45/45, Loss: 0.3820
Train Loss: 0.3603, Train Acc: 0.8903
Test Loss: 0.3390, Test Acc: 0.8955
LR: 0.001000, Time: 776.9s
Best Test Acc: 0.8955
------------------------------------------------------------
Epoch 4/10
  Batch 15/45, Loss: 0.3965
  Batch 30/45, Loss: 0.2657
  Batch 45/45, Loss: 0.2238
Train Loss: 0.3046, Train Acc: 0.9062
Test Loss: 0.2721, Test Acc: 0.9181
LR: 0.001000, Time: 623.5s
Best Test Acc: 0.9181
----------------------------------------